## 3.1 决策树的构造

### 3.1.1 信息增益

In [ ]:
from math import log
import operator

In [4]:
def createDataSet():
    """创建数据集"""
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    #change to discrete values
    return dataSet, labels

def calcShannonEnt(dataSet):
    """计算给定数据集的香农熵"""
    # 数据集实例数
    numEntries = len(dataSet)
    labelCounts = {}
    # 记录每个类别出现的次数
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    # 计算香农熵
    shannonEnt = 0.0
    for key in labelCounts:
        # 选择该分类的概率
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob, 2) 
    return shannonEnt

myDat, labels = createDataSet()
calcShannonEnt(myDat)

0.9709505944546686

In [8]:
# 熵越高，则混合的数据也越多，我们可以在数据集中添加更多的分类，
myDat[0][-1]='maybe'
calcShannonEnt(myDat)

1.3709505944546687

### 3.1.2 划分数据集

In [19]:
def splitDataSet(dataSet, axis, value):
    """按照给定特征划分数据集
    
    :param dataSet: 待划分的数据集.
    :param axis: 划分数据集的特征.
    :param value: 需要返回的特征的值.
    :return: : retDataSet
    """
    # 新建返回数据集
    retDataSet = []
    # 遍历数据集
    for featVec in dataSet:
        # 抽取符合特征的数据
        if featVec[axis] == value:
            # 删除axis特征
            reducedFeatVec = featVec[:axis]    
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

myDat, labels = createDataSet()
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [20]:
splitDataSet(myDat, 1, 0)

[[1, 'no']]

In [21]:
# 在函数中调用的数据需要满足一定的要求
# 第一个要求是，数据必须是一种由列表元素组成的列表，而且所有的列表元素都要具有相同的数据长度；
# 第二个要求是，数据的最后一列或者每个实例的最后一个元素是当前实例的类别标签。
def chooseBestFeatureToSplit(dataSet):
    """选择最好的数据集划分方式"""
    
    numFeatures = len(dataSet[0]) - 1      #the last column is used for the labels
    # 计算数据集的原始香农熵
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; 
    bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        featList = [example[i] for example in dataSet]#create a list of all the examples of this feature
        uniqueVals = set(featList)       #get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature                      #returns an integer